In [1]:
import geopandas as gpd
import h3
import numpy as np
import pandas as pd
from shapely.geometry import Polygon

In [2]:
CITY = "budapest"
VERSION = "20250428"

In [3]:
districts = gpd.read_file(
    "../../data/osm/budapest/budapest_districts_without_margaret_island.geojson"
)

In [4]:
OUTPUT = f"../../output/{CITY}/{VERSION}"

# Read the GeoJSON file where for each stop id we know the multimodal area (5 min walk + 10 min BKK) + ellipticity of this shape + size of the area
# and multipolygons describing the 5 min walk from each such station
multimodal = gpd.read_file(
    f"{OUTPUT}/stop_geometries_from_walk.geojson", engine="pyogrio"
)
# Read GeoJSON file for 15 minute walking distance
walking = gpd.read_file(f"{OUTPUT}/isochrones.geojson", engine="pyogrio")
# read centrality csv
centrality = pd.read_csv(f"{OUTPUT}/merged.csv", dtype={"stop_id": str})

# read in geojson for city boundary
bp = gpd.read_file(f"../../data/osm/{CITY}/boundary.geojson", engine="pyogrio")
# Read the pickle with telekom data for socioecon info
szk_nap = pd.read_pickle("../../data/telekom/szuperkupa_nap_zscores.pkl")
# income = pd.read_csv("/mnt/common-ssd/zadorzsofi/telekom/BKK/data/socioecon/stadat-jov0003-14.8.1.2_income_deciles_total.csv", header=1, index_col=0, delimiter=";", thousands=" ")
# distrcit gross income
dist_income = pd.read_csv(f"../../data/statistics/{CITY}/gross_mean_income_2024.csv")
# gini from housing price
gini_house_multi = pd.read_csv(f"{OUTPUT}/multimodal_stop_gini.csv")
gini_house_walk = pd.read_csv(f"{OUTPUT}/walk15_stop_gini.csv")

In [5]:
szk_nap

,raster_id,traffic,remainers,loc_home,loc_work,loc_freq,sex_female,sex_male,sex_na,arpu_low,...,log_roam_mean,log_roam_std,log_local_mean,log_local_std,log_traffic_z_score_full,log_roam_z_score_full,log_local_z_score_full,log_traffic_z_score,log_roam_z_score,log_local_z_score
87,8a1e0378c367fff,156.000000,46.428571,0.000000,6.428571,1.714286,46.571429,41.000000,68.428571,86.857143,...,-11.512925,0.000000,4.737667,0.455096,0.685984,0.000000,0.685984,0.766493,0.000000,0.766493
255,8a1e037a85b7fff,365.714286,100.428571,23.428571,22.571429,11.428571,99.571429,103.571429,162.571429,183.000000,...,2.318035,0.260093,5.690899,0.287360,0.629836,1.815712,0.575585,0.698256,3.227631,0.633760
423,8a1e0378dd9ffff,91.285714,27.000000,0.000000,5.000000,1.428571,25.571429,24.428571,41.285714,44.857143,...,-11.512925,0.000000,4.312927,0.416590,0.482651,0.000000,0.482651,0.526083,0.000000,0.526083
591,8a1e0363298ffff,65.714286,20.428571,2.142857,0.000000,0.000000,20.285714,18.428571,17.428571,37.714286,...,-11.512925,0.000000,4.164371,0.219665,0.095352,0.000000,0.095352,0.101641,0.000000,0.101641
759,8a1e036ac98ffff,33.857143,2.428571,0.000000,0.000000,0.000000,6.000000,7.285714,6.857143,8.000000,...,-11.512925,0.000000,3.314008,0.356854,0.583268,0.000000,0.583268,0.642815,0.000000,0.642815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4154640,8a1e036ac50ffff,1.428571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-11.512925,0.000000,0.685131,0.294352,-1.115885,0.000000,-1.115886,-1.365136,0.000000,-1.365137
4154684,8a1e036a1c67fff,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-11.512925,0.000000,0.775294,0.457237,0.707107,0.000000,0.707107,0.000000,0.000000,0.000000
4154698,8a1e036118e7fff,3.428571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-11.512925,0.000000,0.709266,0.461908,1.131989,0.000000,1.131989,6.083641,0.000000,6.083630
4154720,8a1e0373322ffff,5.714286,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-11.512925,0.000000,1.029722,0.413670,1.724191,0.000000,1.724191,7.005363,0.000000,7.005352


In [6]:
# Gross labour income per capita by income deciles for Hungary
income = (
    pd.read_csv(
        "../../data/statistics/hungary/stadat-jov0003-14.8.1.2-en.csv",
        delimiter=";",
        thousands=" ",
        skiprows=1,
        skipfooter=26,
        engine="python",
    )
    .drop(0)
    .reset_index(drop=True)
)
income.set_index("Denomination", inplace=True)

# Pipeline

0. Filter walking to 15min walk
1. match hexagons & other small shapes to walking15 shapes and multimodal
2. Append to socioecon data
3. For multimodal, count average income, nr poor households/total households (poor household %) within area
4. For walking15 count within area
5. Calculate a) low income ratio b) Gini, c) income entropy -- however how? income level at each location? and characterise low/mid/high income for Gini?

### Filter walking

In [7]:
walking15 = walking[(walking["costing"] == "walk") & (walking["range"] == 15)]
walking15 = walking15.copy()
walking15.drop(columns=["costing", "range"], inplace=True)

### 1. Match hex to shapes

In [8]:
def add_hexagon_ids(geo_df, hex_resolution=10):
    """
    Adds a column of H3 hexagon IDs to a GeoDataFrame based on its geometries.

    Parameters:
    geo_df (GeoDataFrame): The input GeoDataFrame with geometries.
    hex_resolution (int): The H3 resolution level to use.

    Returns:
    GeoDataFrame: The GeoDataFrame with a new column 'hexagon_ids' containing the hexagon IDs.
    """
    hexagon_lists = []  # List to store hexagon IDs for each geometry

    # Loop through each geometry in the GeoDataFrame
    for idx, row in geo_df.iterrows():
        # Ensure the geometry is valid
        if row.geometry.is_valid:
            # Initialize an empty set to collect hex IDs
            hex_ids = set()

            # Check if the geometry is a MultiPolygon
            if row.geometry.geom_type == "MultiPolygon":
                # Loop through each Polygon in the MultiPolygon
                for polygon in row.geometry.geoms:
                    geom_dict = polygon.__geo_interface__  # Get GeoJSON representation
                    hex_ids.update(
                        h3.polyfill(geom_dict, hex_resolution, geo_json_conformant=True)
                    )
            else:
                # Handle Polygon geometries
                geom_dict = row.geometry.__geo_interface__  # Get GeoJSON representation
                hex_ids.update(
                    h3.polyfill(geom_dict, hex_resolution, geo_json_conformant=True)
                )

            # Add the hex IDs to the list
            hexagon_lists.append(list(hex_ids))
        else:
            # Add an empty list for invalid geometries
            hexagon_lists.append([])

    # Add the hexagon IDs as a new column to the GeoDataFrame
    geo_df["hexagon_ids"] = hexagon_lists

    # Drop rows where 'hexagon_ids' column is empty or contains NaN
    geo_df = geo_df[geo_df["hexagon_ids"].apply(lambda x: bool(x))]

In [9]:
# match hexagons to 15min walking areas
add_hexagon_ids(walking15, hex_resolution=10)
# Explode the hexagon_ids column in the walking GeoDataFrame
walking15_exploded = walking15.explode("hexagon_ids").rename(
    columns={"hexagon_ids": "h3_id"}
)

In [10]:
# match hexagons to Multimodal areas
add_hexagon_ids(multimodal, hex_resolution=10)
# Explode the hexagon_ids column in the walking GeoDataFrame
multimodal_exploded = multimodal.explode("hexagon_ids").rename(
    columns={"hexagon_ids": "h3_id"}
)

### # 2. Append to socioecon data

#### 0. Assign district level gross mean income to each hex in the district

In [11]:
# Merge the dataframes based on the district number
districts3 = pd.merge(
    districts,
    dist_income[["district", "gross_mean_income_2024"]],
    left_on="district_number",
    right_on="district",
    how="left",
)


def h3_to_polygon(h3_id):
    """Convert a H3 hexagon ID to a Shapely Polygon."""
    # Get the vertices of the hexagon using h3 library
    boundary = h3.h3_to_geo_boundary(h3_id, geo_json=True)
    # Convert the list of boundary points into a Polygon
    return Polygon(boundary)


# Apply the conversion for each raster_id in szk_nap
szk_nap["geometry"] = szk_nap["raster_id"].apply(h3_to_polygon)

# Now create a GeoDataFrame
szk_nap_gdf = gpd.GeoDataFrame(szk_nap, geometry="geometry", crs="EPSG:4326")

# Perform a spatial join to map hexagons to districts
szk_nap_gdf2 = gpd.sjoin(szk_nap_gdf, districts3, how="left")

szk_nap_gdf3 = szk_nap_gdf2.drop(
    columns=["geometry", "index_right", "name", "short_name", "district_number"]
)

In [12]:
szk_nap_gdf3.head()

,raster_id,traffic,remainers,loc_home,loc_work,loc_freq,sex_female,sex_male,sex_na,arpu_low,...,log_local_std,log_traffic_z_score_full,log_roam_z_score_full,log_local_z_score_full,log_traffic_z_score,log_roam_z_score,log_local_z_score,osm_id,district,gross_mean_income_2024
87,8a1e0378c367fff,156.000000,46.428571,0.000000,6.428571,1.714286,46.571429,41.000000,68.428571,86.857143,...,0.455096,0.685984,0.000000,0.685984,0.766493,0.000000,0.766493,1606100.0,8.0,771000.0
255,8a1e037a85b7fff,365.714286,100.428571,23.428571,22.571429,11.428571,99.571429,103.571429,162.571429,183.000000,...,0.287360,0.629836,1.815712,0.575585,0.698256,3.227631,0.633760,1606101.0,6.0,847000.0
423,8a1e0378dd9ffff,91.285714,27.000000,0.000000,5.000000,1.428571,25.571429,24.428571,41.285714,44.857143,...,0.416590,0.482651,0.000000,0.482651,0.526083,0.000000,0.526083,1606100.0,8.0,771000.0
591,8a1e0363298ffff,65.714286,20.428571,2.142857,0.000000,0.000000,20.285714,18.428571,17.428571,37.714286,...,0.219665,0.095352,0.000000,0.095352,0.101641,0.000000,0.101641,1552463.0,10.0,698000.0
759,8a1e036ac98ffff,33.857143,2.428571,0.000000,0.000000,0.000000,6.000000,7.285714,6.857143,8.000000,...,0.356854,0.583268,0.000000,0.583268,0.642815,0.000000,0.642815,1550597.0,23.0,688000.0


In [13]:
# Merge 15 min walk with df_szk on the H3 hexagon IDs
stop_walk15 = pd.merge(
    walking15_exploded, szk_nap_gdf3, left_on="h3_id", right_on="raster_id", how="left"
)

In [14]:
# Merge with df_szk on the H3 hexagon IDs
stop_multimodal = pd.merge(
    multimodal_exploded, szk_nap_gdf2, left_on="h3_id", right_on="raster_id", how="left"
)

### 3. and 4. Get socioecon information for each multimodal and walking area

In [15]:
# Define the columns to sum and the column to average
columns_to_sum = [
    "traffic",
    "sex_female",
    "sex_male",
    "sex_na",
    "arpu_low",
    "arpu_mid",
    "arpu_high",
    "arpu_na",
    "age_young",
    "age_mid",
    "age_old",
]

column_to_avg = "gross_mean_income_2024"

# For 15-min walking area
nr_ppl_per_stop_walk15 = (
    stop_walk15.groupby("stop_id")
    .agg(
        {
            **{col: "sum" for col in columns_to_sum},  # Sum columns
            column_to_avg: "mean",  # Average for income - so it takes into account if area is in several districts
        }
    )
    .reset_index()
)

# For multimodal area
nr_ppl_per_stop_multimodal = (
    stop_multimodal.groupby("stop_id")
    .agg(
        {
            **{col: "sum" for col in columns_to_sum},  # Sum  columns
            column_to_avg: "mean",  # Average for 'Gross mean income monthly 2024'
        }
    )
    .reset_index()
)

### Calculate socio-econ changes

#### a) Income entropy

In [16]:
# nr_ppl_per_stop_multimodal2 = nr_ppl_per_stop_multimodal.set_index("stop_id")
# multimodal_rev = nr_ppl_per_stop_multimodal2.drop(
#     columns=[
#         "traffic",
#         "sex_female",
#         "sex_male",
#         "sex_na",
#         "age_young",
#         "age_mid",
#         "age_old",
#         "arpu_high",
#         "gross_mean_income_2024",
#     ]
# )

In [17]:
# multimodal_rev["entropy"] = multimodal_rev.apply(
#     lambda row: entropy(row.values, 3), axis=1
# )

In [18]:
# nr_ppl_per_stop_multimodal = nr_ppl_per_stop_multimodal.merge(
#     multimodal_rev[["entropy"]],
#     left_on="stop_id",
#     right_on=multimodal_rev.index,
#     how="left",
# )

In [19]:
# nr_ppl_per_stop_walk15_2 = nr_ppl_per_stop_walk15.set_index("stop_id")
# walk15_rev = nr_ppl_per_stop_walk15_2.drop(
#     columns=[
#         "traffic",
#         "sex_female",
#         "sex_male",
#         "sex_na",
#         "age_young",
#         "age_mid",
#         "age_old",
#         "arpu_high",
#         "gross_mean_income_2024",
#     ]
# )

In [20]:
# walk15_rev["entropy"] = walk15_rev.apply(lambda row: entropy(row.values, 3), axis=1)
# nr_ppl_per_stop_walk15 = nr_ppl_per_stop_walk15.merge(
#     walk15_rev[["entropy"]], left_on="stop_id", right_on=walk15_rev.index, how="left"
# )

#### b) Revenue index

In [21]:
# # Generate Weighted average for multimodal areas

# # Calculate the total number of people at each stop
# nr_ppl_per_stop_multimodal["total"] = (
#     nr_ppl_per_stop_multimodal["arpu_low"]
#     + nr_ppl_per_stop_multimodal["arpu_mid"]
#     + nr_ppl_per_stop_multimodal["arpu_high"]
# )

# # Calculate the proportion of each category
# nr_ppl_per_stop_multimodal["P_low"] = (
#     nr_ppl_per_stop_multimodal["arpu_low"] / nr_ppl_per_stop_multimodal["total"]
# )
# nr_ppl_per_stop_multimodal["P_mid"] = (
#     nr_ppl_per_stop_multimodal["arpu_mid"] / nr_ppl_per_stop_multimodal["total"]
# )
# nr_ppl_per_stop_multimodal["P_high"] = (
#     nr_ppl_per_stop_multimodal["arpu_high"] / nr_ppl_per_stop_multimodal["total"]
# )

# # Calculate the revenue index using weights -1, 0, and 1
# nr_ppl_per_stop_multimodal["revenue_index"] = (
#     (-1 * nr_ppl_per_stop_multimodal["P_low"])
#     + (0 * nr_ppl_per_stop_multimodal["P_mid"])
#     + (1 * nr_ppl_per_stop_multimodal["P_high"])
# )

In [22]:
# # Generate Weighted average for multimodal areas

# # Calculate the total number of people at each stop
# nr_ppl_per_stop_walk15["total"] = (
#     nr_ppl_per_stop_walk15["arpu_low"]
#     + nr_ppl_per_stop_walk15["arpu_mid"]
#     + nr_ppl_per_stop_walk15["arpu_high"]
# )

# # Calculate the proportion of each category
# nr_ppl_per_stop_walk15["P_low"] = (
#     nr_ppl_per_stop_walk15["arpu_low"] / nr_ppl_per_stop_walk15["total"]
# )
# nr_ppl_per_stop_walk15["P_mid"] = (
#     nr_ppl_per_stop_walk15["arpu_mid"] / nr_ppl_per_stop_walk15["total"]
# )
# nr_ppl_per_stop_walk15["P_high"] = (
#     nr_ppl_per_stop_walk15["arpu_high"] / nr_ppl_per_stop_walk15["total"]
# )

# # Calculate the revenue index using weights -1, 0, and 1
# nr_ppl_per_stop_walk15["revenue_index"] = (
#     (-1 * nr_ppl_per_stop_walk15["P_low"])
#     + (0 * nr_ppl_per_stop_walk15["P_mid"])
#     + (1 * nr_ppl_per_stop_walk15["P_high"])
# )

#### c) Gini coefficient

##### 1. Calculate % difference from Hungary income deciles to low-income, medium income and high income

In [23]:
# For Helsinki I used these deciles, so I'll continue using them for consistency

# In the postal code data low-income is nr of people in decile 1 and 2
# High-income is nr of people in decile 9-10
# So I want to calculate % difference in income from median for low and high income groups and use that in the GINI

In [24]:
# Compute Bp's Income Differences between median (5th decile) and low/high income
median_bp = income.loc[income.index == "5th income decile", "2020"]

low_deciles_avg = income.loc[
    income.index.isin(["1st income decile", "2nd income decile"]), "2020"
].mean()
high_deciles_avg = income.loc[
    income.index.isin(["9th income decile", "10th income decile"]), "2020"
].mean()

low_income_diff = (low_deciles_avg - median_bp) / median_bp
high_income_diff = (high_deciles_avg - median_bp) / median_bp

In [25]:
# Estimate Representative Incomes for walking
nr_ppl_per_stop_walk15["Low_Income_Rep"] = nr_ppl_per_stop_walk15[
    "gross_mean_income_2024"
] * (1 + float(low_income_diff.iloc[0]))
nr_ppl_per_stop_walk15["High_Income_Rep"] = nr_ppl_per_stop_walk15[
    "gross_mean_income_2024"
] * (1 + float(high_income_diff.iloc[0]))

# Estimate Representative Incomes for multimodal
nr_ppl_per_stop_multimodal["Low_Income_Rep"] = nr_ppl_per_stop_multimodal[
    "gross_mean_income_2024"
] * (1 + float(low_income_diff.iloc[0]))
nr_ppl_per_stop_multimodal["High_Income_Rep"] = nr_ppl_per_stop_multimodal[
    "gross_mean_income_2024"
] * (1 + float(high_income_diff.iloc[0]))

##### 2. Apply % difference to walking and multimodal areas

In [26]:
# creating an array (column), where each income category is repeated the houshold nr times
# this will be used for gini calculation
def create_income_distribution(row):
    low_income = [row["Low_Income_Rep"]] * int(row["arpu_low"])
    middle_income = [row["gross_mean_income_2024"]] * int(row["arpu_mid"])
    high_income = [row["High_Income_Rep"]] * int(row["arpu_high"])
    return low_income + middle_income + high_income

In [27]:
# define gini
def gini(array):
    """Calculate the Gini coefficient of a numpy array.
    Based on code from: https://github.com/oliviaguest/gini"""
    array = np.array(
        array, dtype=float
    )  # change from Olivia: Convert to NumPy array and flatten
    if (
        array.size == 0 or np.sum(array) == 0
    ):  # change from Olivia, as I have empty arrays
        return np.nan  # handle 0 divisions
    array = np.sort(array)
    n = array.size
    index = np.arange(1, n + 1)
    return (np.sum((2 * index - n - 1) * array)) / (n * np.sum(array))

In [28]:
nr_ppl_per_stop_walk15["Income_Distribution"] = nr_ppl_per_stop_walk15.apply(
    create_income_distribution, axis=1
)
nr_ppl_per_stop_walk15["gini"] = nr_ppl_per_stop_walk15["Income_Distribution"].apply(
    gini
)

In [29]:
nr_ppl_per_stop_multimodal["Income_Distribution"] = nr_ppl_per_stop_multimodal.apply(
    create_income_distribution, axis=1
)
nr_ppl_per_stop_multimodal["gini"] = nr_ppl_per_stop_multimodal[
    "Income_Distribution"
].apply(gini)

##### 2. Merge Gini from housing prices

In [30]:
# Rename column in gini_house_multi
gini_house_multi = gini_house_multi.rename(columns={"multimodal_gini": "gini_house"})

# Rename column in gini_house_walk
gini_house_walk = gini_house_walk.rename(columns={"walk15_gini": "gini_house"})

# Merge for multimodal
nr_ppl_per_stop_multimodal = pd.merge(
    gini_house_multi,
    nr_ppl_per_stop_multimodal,
    on="stop_id",
    how="left",  # or 'inner' depending on your goal
)

# Merge for walk
nr_ppl_per_stop_walk15 = pd.merge(
    gini_house_walk, nr_ppl_per_stop_walk15, on="stop_id", how="left"
)

#### Calculate socioecon ratios

In [31]:
nr_ppl_per_stop_walk15["fem_ratio"] = (
    nr_ppl_per_stop_walk15["sex_female"] / nr_ppl_per_stop_walk15["traffic"]
)
nr_ppl_per_stop_walk15["arpu_low_ratio"] = (
    nr_ppl_per_stop_walk15["arpu_low"] / nr_ppl_per_stop_walk15["traffic"]
)
nr_ppl_per_stop_walk15["arpu_high_ratio"] = (
    nr_ppl_per_stop_walk15["arpu_high"] / nr_ppl_per_stop_walk15["traffic"]
)
nr_ppl_per_stop_walk15["young_ratio"] = (
    nr_ppl_per_stop_walk15["age_young"] / nr_ppl_per_stop_walk15["traffic"]
)
nr_ppl_per_stop_walk15["old_ratio"] = (
    nr_ppl_per_stop_walk15["age_old"] / nr_ppl_per_stop_walk15["traffic"]
)

In [32]:
nr_ppl_per_stop_multimodal["fem_ratio"] = (
    nr_ppl_per_stop_multimodal["sex_female"] / nr_ppl_per_stop_multimodal["traffic"]
)
nr_ppl_per_stop_multimodal["arpu_low_ratio"] = (
    nr_ppl_per_stop_multimodal["arpu_low"] / nr_ppl_per_stop_multimodal["traffic"]
)
nr_ppl_per_stop_multimodal["arpu_high_ratio"] = (
    nr_ppl_per_stop_multimodal["arpu_high"] / nr_ppl_per_stop_multimodal["traffic"]
)
nr_ppl_per_stop_multimodal["young_ratio"] = (
    nr_ppl_per_stop_multimodal["age_young"] / nr_ppl_per_stop_multimodal["traffic"]
)
nr_ppl_per_stop_multimodal["old_ratio"] = (
    nr_ppl_per_stop_multimodal["age_old"] / nr_ppl_per_stop_multimodal["traffic"]
)

### Merge different datasets

In [33]:
# Merge the DataFrames on stop_id with suffixes but remove Income_Dist before merging as I won't need this in the final df
nr_ppl_per_stop_comparison = pd.merge(
    nr_ppl_per_stop_multimodal.drop(columns=["Income_Distribution"]),
    nr_ppl_per_stop_walk15.drop(columns=["Income_Distribution"]),
    on="stop_id",
    suffixes=("_multimodal", "_walk15"),
)

In [34]:
# calculate change of some variables
nr_ppl_comparison = nr_ppl_per_stop_comparison.dropna()
nr_ppl_comparison = nr_ppl_comparison.copy()

nr_ppl_comparison.loc[:, "fem_change"] = (
    nr_ppl_comparison["fem_ratio_multimodal"] - nr_ppl_comparison["fem_ratio_walk15"]
)
nr_ppl_comparison.loc[:, "arpu_low_change"] = (
    nr_ppl_comparison["arpu_low_ratio_multimodal"]
    - nr_ppl_comparison["arpu_low_ratio_walk15"]
)
nr_ppl_comparison.loc[:, "young_change"] = (
    nr_ppl_comparison["young_ratio_multimodal"]
    - nr_ppl_comparison["young_ratio_walk15"]
)
nr_ppl_comparison.loc[:, "old_change"] = (
    nr_ppl_comparison["old_ratio_multimodal"] - nr_ppl_comparison["old_ratio_walk15"]
)

In [37]:
# Calculate percentage change
nr_ppl_comparison["percent_change_gini"] = np.where(
    nr_ppl_comparison["gini_walk15"] == 0,
    np.nan,  # because division by 0 is not meaningful, or can be flag value as there is more inequality in the multimodal -- what should be the value?
    (nr_ppl_comparison["gini_multimodal"] - nr_ppl_comparison["gini_walk15"])
    / nr_ppl_comparison["gini_walk15"]
    * 100,
)

nr_ppl_comparison["percent_change_house_gini"] = np.where(
    nr_ppl_comparison["gini_house_walk15"] == 0,
    np.nan,  # because division by 0 is not meaningful, or can be flag value as there is more inequality in the multimodal -- what should be the value?
    (nr_ppl_comparison["gini_house_multimodal"] - nr_ppl_comparison["gini_walk15"])
    / nr_ppl_comparison["gini_house_walk15"]
    * 100,
)

# nr_ppl_comparison["percent_change_rev_index"] = (
#     (
#         nr_ppl_comparison["revenue_index_multimodal"]
#         - nr_ppl_comparison["revenue_index_walk15"]
#     )
#     / nr_ppl_comparison["revenue_index_walk15"]
# ) * 100

# nr_ppl_comparison["percent_change_entropy"] = (
#     (nr_ppl_comparison["entropy_multimodal"] - nr_ppl_comparison["entropy_walk15"])
#     / nr_ppl_comparison["entropy_walk15"]
# ) * 100

In [39]:
# Gini log change
nr_ppl_comparison["log_change_gini"] = np.where(
    (nr_ppl_comparison["gini_multimodal"] > 0) & (nr_ppl_comparison["gini_walk15"] > 0),
    np.log(nr_ppl_comparison["gini_multimodal"] / nr_ppl_comparison["gini_walk15"]),
    np.nan,
)

# Gini (households) log change
nr_ppl_comparison["log_change_house_gini"] = np.where(
    (nr_ppl_comparison["gini_house_multimodal"] > 0)
    & (nr_ppl_comparison["gini_house_walk15"] > 0),
    np.log(
        nr_ppl_comparison["gini_house_multimodal"]
        / nr_ppl_comparison["gini_house_walk15"]
    ),
    np.nan,
)

# # Revenue index log change
# nr_ppl_comparison["log_change_rev_index"] = np.where(
#     (nr_ppl_comparison["revenue_index_multimodal"] > 0)
#     & (nr_ppl_comparison["revenue_index_walk15"] > 0),
#     np.log(
#         nr_ppl_comparison["revenue_index_multimodal"]
#         / nr_ppl_comparison["revenue_index_walk15"]
#     ),
#     np.nan,
# )

# # Entropy log change
# nr_ppl_comparison["log_change_entropy"] = np.where(
#     (nr_ppl_comparison["entropy_multimodal"] > 0)
#     & (nr_ppl_comparison["entropy_walk15"] > 0),
#     np.log(
#         nr_ppl_comparison["entropy_multimodal"] / nr_ppl_comparison["entropy_walk15"]
#     ),
#     np.nan,
# )

/home/gergo/.cache/pypoetry/virtualenvs/calculate_accessibility-4Q_DyPin-py3.11/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/gergo/.cache/pypoetry/virtualenvs/calculate_accessibility-4Q_DyPin-py3.11/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [40]:
centrality.drop_duplicates(subset=["stop_id"], inplace=True)

In [41]:
# Merge the DataFrames
nr_ppl_comparison2 = pd.merge(nr_ppl_comparison, centrality, on="stop_id", how="inner")

In [ ]:
nr_ppl_comparison2.to_csv(f"../../output/{CITY}/bp_socioecon_merged5b.csv", index=False)